[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/4.3%20ETF%20%EB%B6%84%EC%84%9D%EA%B3%BC%20%EC%8B%9C%EA%B0%81%ED%99%94-input.ipynb)


## ETF 분석과 시각화

* keyword
    * seaborn - barplot, boxplot, violinplot
    * **seaborn PairGrid vs matplotlib - subplots**
    * seaborn FacetGrid
        * **catplot 범주형 데이터 시각화 (kind: count, bar, box, boxen, violin)**
        * relplot 연속된 수치 데이터 시각화 (kind: scatter, line) <= KRX 전체 종목 분석에서 다룸
    * pandas - value_counts, cross_tab, groupby, pivot_table
  
<img src="https://seaborn.pydata.org/_images/function_overview_8_0.png">

* seaborn 에 대해 좀 더 알고 싶다면 다음의 튜토리얼을 참고해 보세요.
* [User guide and tutorial — seaborn documentation](https://seaborn.pydata.org/tutorial.html)



* 특정 시점의 데이터를 저장해 분석합니다.
* 분석하는 시점에 따라 내용이 달라지는 점에 유의하세요!
* 내가 수집한 시점의 데이터를 분석하기 때문에 매일 분석한다면 일자별 리포트를 만들어 볼 수도 있을거에요.
* [ETF : 네이버 금융](https://finance.naver.com/sise/etf.nhn)


## ⚠️ 주의사항 ⚠️ 
*  분석내용은 데이터 수집 시점에 따라 달라집니다.

## 라이브러리 로드

In [149]:
# 필요한 도구를 로드합니다.
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [150]:
# seaborn 의 버전을 확인합니다. 
# 0.11.0 이상의 버전에서 변경된 내용이 많으니 0.11 이상의 버전을 사용해 주세요.
sns.__version__

'0.11.2'

In [151]:
pd.__version__

'1.3.5'

## 시각화를 위한 한글폰트 설정

In [152]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

In [153]:
# 사용가능한 그래프 스타일
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [154]:
# 운영체제에 맞는 font_family 찾기 
get_font_family()

/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


'NanumBarunGothic'

In [155]:
# 시각화를 위한 폰트설정
import matplotlib.pyplot as plt

# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family 라는 변수에 할당
font_family = get_font_family()
# 폰트설정
plt.rc("font", family=font_family)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# 그래프 스타일 설정
plt.style.use("ggplot")

# 그래프에 retina display 적용
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [156]:
# 한글폰트 확인
pd.Series([1, 3, 5, -7, 9]).plot(title="한글폰트")

## 데이터 로드

In [157]:
# 이전 수업에서 수집해서 저장해둔 csv 파일을 읽어옵니다.
# itemcode 숫자 앞의 0 이 지워진다면 
# dtype={"종목코드": np.object} 로 타입을 지정해 주면 문자형태로 읽어옵니다.
# df
from google.colab import drive

drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/Colab Notebooks/IBK투자증권/finance-data-analysis-main"
df = pd.read_csv(f"{path}/eft_2022-04-28.csv", dtype={"itemcode": np.object})
df

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


,종목코드,탭코드,종목명,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억),유형,브랜드,인버스,레버리지,환헤지H
0,69500,1,KODEX 200,34985,2,345,1.00,35084.0,-0.4674,4180043,145311,47195,국내 시장지수,KODEX,False,False,False
1,371460,4,TIGER 차이나전기차SOLACTIVE,11890,2,95,0.81,11894.0,-25.5132,8956866,107221,29425,해외 주식,TIGER,False,False,False
2,122630,3,KODEX 레버리지,18585,2,360,1.98,18679.0,-2.6183,16856345,309205,21122,국내 파생,KODEX,False,True,False
3,133690,4,TIGER 미국나스닥100,75220,2,1000,1.35,74008.0,-0.9795,117485,8789,20964,해외 주식,TIGER,False,False,False
4,252670,3,KODEX 200선물인버스2X,2650,5,-50,-1.85,2646.0,-1.0990,141997440,380833,19202,국내 파생,KODEX,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,295000,6,KBSTAR 국채선물10년,51165,5,-10,-0.02,51181.0,-3.7159,39,1,20,채권,KBSTAR,False,False,False
551,287330,2,KBSTAR 200생활소비재,7395,2,255,3.57,7392.0,6.9597,485,3,18,국내 업종/테마,KBSTAR,False,False,False
552,284990,2,KBSTAR 200에너지화학,11195,2,290,2.66,11214.0,1.1791,316,3,16,국내 업종/테마,KBSTAR,False,False,False
553,287310,2,KBSTAR 200경기소비재,10060,2,100,1.00,10079.0,2.7763,380,3,10,국내 업종/테마,KBSTAR,False,False,False


In [158]:
# 인덱스 번호 상단 5개의 데이터를 가져옵니다.
# 제대로 read되었는지 각 column정보들을 확인합니다.
df.head()

,종목코드,탭코드,종목명,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억),유형,브랜드,인버스,레버리지,환헤지H
0,69500,1,KODEX 200,34985,2,345,1.00,35084.0,-0.4674,4180043,145311,47195,국내 시장지수,KODEX,False,False,False
1,371460,4,TIGER 차이나전기차SOLACTIVE,11890,2,95,0.81,11894.0,-25.5132,8956866,107221,29425,해외 주식,TIGER,False,False,False
2,122630,3,KODEX 레버리지,18585,2,360,1.98,18679.0,-2.6183,16856345,309205,21122,국내 파생,KODEX,False,True,False
3,133690,4,TIGER 미국나스닥100,75220,2,1000,1.35,74008.0,-0.9795,117485,8789,20964,해외 주식,TIGER,False,False,False
4,252670,3,KODEX 200선물인버스2X,2650,5,-50,-1.85,2646.0,-1.0990,141997440,380833,19202,국내 파생,KODEX,True,False,False


In [159]:
# 인덱스 하단의 5개 데이터를 가져옵니다.
# 데이터가 제대로 로드되었는지 확인합니다.
df.tail()

,종목코드,탭코드,종목명,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억),유형,브랜드,인버스,레버리지,환헤지H
550,295000,6,KBSTAR 국채선물10년,51165,5,-10,-0.02,51181.0,-3.7159,39,1,20,채권,KBSTAR,False,False,False
551,287330,2,KBSTAR 200생활소비재,7395,2,255,3.57,7392.0,6.9597,485,3,18,국내 업종/테마,KBSTAR,False,False,False
552,284990,2,KBSTAR 200에너지화학,11195,2,290,2.66,11214.0,1.1791,316,3,16,국내 업종/테마,KBSTAR,False,False,False
553,287310,2,KBSTAR 200경기소비재,10060,2,100,1.00,10079.0,2.7763,380,3,10,국내 업종/테마,KBSTAR,False,False,False
554,287320,2,KBSTAR 200산업재,11760,2,240,2.08,11784.0,18.4250,37,0,7,국내 업종/테마,KBSTAR,False,False,False


In [160]:
# info를 통해서 각 column들의 데이터타입과 결측치, 메모리 사용량 등을 볼 수 있습니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   종목코드        555 non-null    int64  
 1   탭코드         555 non-null    int64  
 2   종목명         555 non-null    object 
 3   현재가         555 non-null    int64  
 4   등락구분        555 non-null    int64  
 5   전일비         555 non-null    int64  
 6   등락률         555 non-null    float64
 7   순자산가치(NAV)  555 non-null    float64
 8   3개월수익률      537 non-null    float64
 9   거래량         555 non-null    int64  
 10  거래대금(백만)    555 non-null    int64  
 11  시가총액(억)     555 non-null    int64  
 12  유형          555 non-null    object 
 13  브랜드         555 non-null    object 
 14  인버스         555 non-null    bool   
 15  레버리지        555 non-null    bool   
 16  환헤지H        555 non-null    bool   
dtypes: bool(3), float64(3), int64(8), object(3)
memory usage: 62.5+ KB


## 결측치 확인

In [161]:
(True + True + False) / 3

0.6666666666666666

In [162]:
# 결측치를 확인합니다.
df.isnull().sum()

종목코드           0
탭코드            0
종목명            0
현재가            0
등락구분           0
전일비            0
등락률            0
순자산가치(NAV)     0
3개월수익률        18
거래량            0
거래대금(백만)       0
시가총액(억)        0
유형             0
브랜드            0
인버스            0
레버리지           0
환헤지H           0
dtype: int64

In [163]:
# 결측치의 비율을 봅니다.
df.isnull().mean() *100

종목코드          0.000000
탭코드           0.000000
종목명           0.000000
현재가           0.000000
등락구분          0.000000
전일비           0.000000
등락률           0.000000
순자산가치(NAV)    0.000000
3개월수익률        3.243243
거래량           0.000000
거래대금(백만)      0.000000
시가총액(억)       0.000000
유형            0.000000
브랜드           0.000000
인버스           0.000000
레버리지          0.000000
환헤지H          0.000000
dtype: float64

In [164]:
# heatmap을 통한 결측치 시각화
sns.heatmap(df.isnull(),cmap="Greys_r")

## 범주형 데이터의 빈도수
### 범주형 변수 찾기

In [165]:
# describe를 통해 범주형 변수에 대한 기술통계 값을 구하기
df.describe()

,종목코드,탭코드,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억)
count,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,537.000000,5.550000e+02,555.000000,555.000000
mean,280200.252252,3.286486,20125.801802,2.774775,92.873874,0.649243,20109.778378,-0.166023,5.404892e+05,4226.300901,1316.410811
std,92203.159462,1.763780,24272.946502,1.292052,175.092246,1.051226,24272.004292,9.537002,6.326138e+06,27054.799120,3737.443096
min,69500.000000,1.000000,2555.000000,2.000000,-715.000000,-2.800000,162.000000,-63.281700,0.000000e+00,0.000000,7.000000
25%,225045.000000,2.000000,8967.500000,2.000000,0.000000,0.000000,8949.000000,-3.636800,2.870000e+02,4.000000,81.000000
50%,284980.000000,3.000000,11705.000000,2.000000,85.000000,0.750000,11707.000000,0.000000,4.659000e+03,58.000000,203.000000
75%,364975.000000,4.000000,16717.500000,3.000000,155.000000,1.240000,16694.500000,4.277100,3.217450e+04,438.500000,808.000000
max,426330.000000,7.000000,111830.000000,5.000000,1750.000000,5.860000,111704.000000,27.976100,1.419974e+08,380833.000000,47195.000000


In [166]:
# unique(중복을 제외한 값)에 대한 빈도수 구하기
# object type 이 아니더라도 nunique 값이 작다면 범주형 변수인지 확인해 봅니다.
df.nunique()

종목코드          555
탭코드             7
종목명           555
현재가           509
등락구분            3
전일비           129
등락률           282
순자산가치(NAV)    544
3개월수익률        535
거래량           500
거래대금(백만)      295
시가총액(억)       388
유형              7
브랜드            21
인버스             2
레버리지            2
환헤지H            2
dtype: int64

### ETF 상품 유형별 빈도수(한 개의 변수)

In [167]:
# df["유형"].value_counts() 의 결과와 같게 시각화 할 수 있습니다.
df["유형"].value_counts()

국내 업종/테마    205
해외 주식       136
국내 시장지수      64
채권           60
국내 파생        37
기타           35
원자재          18
Name: 유형, dtype: int64

In [168]:
# sns.countplot 으로 빈도수를 시각화 합니다.
sns.countplot(data =df["유형"].value_counts())

In [169]:
# value_counts(normalize=True) 로 빈도수의 비율을 구합니다.
df["유형"].value_counts(normalize=True)*100

국내 업종/테마    36.936937
해외 주식       24.504505
국내 시장지수     11.531532
채권          10.810811
국내 파생        6.666667
기타           6.306306
원자재          3.243243
Name: 유형, dtype: float64

### 두 개의 변수 빈도수와 비율

In [170]:
# 유형 컬럼의 빈도수를 봅니다.
# seaborn countplot의 hue를 이용하여 인버스여부에 따라 다른 색상으로 표시해 봅니다.
sns.countplot(data=df, y="유형", hue="인버스")

In [171]:
# pandas crosstab을 이용하여 위 그래프의 수치를 확인 할 수 있습니다.
# seaborn 은 아래와 같은 연산을 하지 않더라도 알아서 통계적 연산을 시각화 해줍니다.
pd.crosstab(df["인버스"], df["유형"])

유형,국내 시장지수,국내 업종/테마,국내 파생,기타,원자재,채권,해외 주식
인버스,,,,,,,
False,64,205,20,30,14,51,130
True,0,0,17,5,4,9,6


In [172]:
# value_counts 처럼 normalize 를 사용해 비율을 구할 수도 있습니다.
n = pd.crosstab(df["인버스"], df["유형"], normalize=True) * 100
n

유형,국내 시장지수,국내 업종/테마,국내 파생,기타,원자재,채권,해외 주식
인버스,,,,,,,
False,11.531532,36.936937,3.603604,5.405405,2.522523,9.189189,23.423423
True,0.000000,0.000000,3.063063,0.900901,0.720721,1.621622,1.081081


In [173]:
# 동일하게 유형 데이터를 레버리지 여부에 따라 그립니다.
sns.countplot(data=df, y="유형", hue="레버리지")

In [174]:
# 레버리지, 유형에 대한 빈도수를 구힙니다.
pd.crosstab(df["레버리지"], df["유형"])

유형,국내 시장지수,국내 업종/테마,국내 파생,기타,원자재,채권,해외 주식
레버리지,,,,,,,
False,64,201,21,32,18,58,123
True,0,4,16,3,0,2,13


In [175]:
# 레버리지, 유형에 대한 빈도수의 비율을 구합니다.
pd.crosstab(df["레버리지"], df["유형"], normalize=True) * 100

유형,국내 시장지수,국내 업종/테마,국내 파생,기타,원자재,채권,해외 주식
레버리지,,,,,,,
False,11.531532,36.216216,3.783784,5.765766,3.243243,10.45045,22.162162
True,0.000000,0.720721,2.882883,0.540541,0.000000,0.36036,2.342342


### 브랜드별 빈도수

In [176]:
# 브랜드별 ETF 빈도수
sns.countplot(data=df, y="브랜드", 
              order=df["브랜드"].value_counts().index)

### 인버스, 브랜드의 ETF 빈도 비율


* 인버스 ETF란? 주식관련 장내 외 파생상품 투자 및 증권차입매도 등을 통해 기초지수(KOSPI 200지수)의 일일 변동률(일별수익률)을 음의 1배수 즉, 역방향으로 추적하는 ETF를 말한다. 예를 들어, KOSPI 200지수가 1% 상승할 경우 인버스 ETF는 마이너스 1% 수익률, 반대로 KOSPI 200 지수가 1% 하락시 인버스 ETF는 플러스 1%의 수익률을 목표로 운영된다.
[네이버 지식백과] 인버스ETF (매일경제, 매경닷컴)
* 출처 : https://terms.naver.com/entry.nhn?docId=20336&cid=43659&categoryId=43659

In [177]:
# 브랜드 column의 빈도수를 인버스에 따른 구분으로 그려봅니다.
sns.countplot(data=df, y="브랜드", hue="인버스")

In [178]:
# 인버스, 브랜드의 금융상품 빈도수
pd.crosstab(df["인버스"], df["브랜드"])

브랜드,ARIRANG,FOCUS,HANARO,HK,KBSTAR,KINDEX,KODEX,KOSEF,KTOP,MASTER,...,TIGER,TIMEFOLIO,TREX,VITA,WOORI,마이다스,마이티,에셋플러스,파워,히어로즈
인버스,,,,,,,,,,,,,,,,,,,,,
False,42,3,28,3,75,51,123,23,1,2,...,132,4,2,1,1,2,2,2,4,1
True,3,0,1,0,10,4,11,5,0,0,...,7,0,0,0,0,0,0,0,0,0


In [179]:
# 인버스, 브랜드의 금융상품 빈도 비율
# in_brand_norm
in_brand_norm = pd.crosstab(df["인버스"], df["브랜드"], normalize=True) * 100
in_brand_norm.round(3)

브랜드,ARIRANG,FOCUS,HANARO,HK,KBSTAR,KINDEX,KODEX,KOSEF,KTOP,MASTER,...,TIGER,TIMEFOLIO,TREX,VITA,WOORI,마이다스,마이티,에셋플러스,파워,히어로즈
인버스,,,,,,,,,,,,,,,,,,,,,
False,7.568,0.541,5.045,0.541,13.514,9.189,22.162,4.144,0.18,0.36,...,23.784,0.721,0.36,0.18,0.18,0.36,0.36,0.36,0.721,0.18
True,0.541,0.000,0.180,0.000,1.802,0.721,1.982,0.901,0.00,0.00,...,1.261,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00


### 레버리지, 브랜드의 ETF 빈도 비율
* 레버리지 ETF란? 일반 ETF가 코스피200과 같은 지수 내 대표 주식을 보유함으로써 지수와 비슷한 수익률을 내는 것을 목표로 하는 데 비해 레버리지ETF는 선물 등 파생상품에 투자해 지수보다 높은 수익을 추구하는 ETF다. 상승장에서는 높은 수익률을 기대할 수 있지만 하락장에서는 손실도 커져 고위험ㆍ고수익 상품으로 분류된다.
[네이버 지식백과] 레버리지 ETF (매일경제, 매경닷컴)
* 출처 : https://terms.naver.com/entry.nhn?docId=2059802&cid=43659&categoryId=43659

In [180]:
# 브랜드별 레버리지 여부에 따른 빈도수를 시각화 합니다.
sns.countplot(data=df, y="브랜드", hue="레버리지")

In [181]:
# 레버리지, 브랜드의 ETF 빈도수(crosstab)
pd.crosstab(df["레버리지"], df["브랜드"])

브랜드,ARIRANG,FOCUS,HANARO,HK,KBSTAR,KINDEX,KODEX,KOSEF,KTOP,MASTER,...,TIGER,TIMEFOLIO,TREX,VITA,WOORI,마이다스,마이티,에셋플러스,파워,히어로즈
레버리지,,,,,,,,,,,,,,,,,,,,,
False,44,3,27,3,81,50,128,24,1,2,...,124,4,2,1,1,2,1,2,4,1
True,1,0,2,0,4,5,6,4,0,0,...,15,0,0,0,0,0,1,0,0,0


In [182]:
# 레버리지, 브랜드의 ETF 빈도수(crosstab)
pd.crosstab(df["레버리지"], df["브랜드"]).T.sort_values(by=False).plot.barh()

In [183]:
# 레버리지, 브랜드의 ETF 비율
pd.crosstab(df["레버리지"], df["브랜드"], normalize=True) * 100

브랜드,ARIRANG,FOCUS,HANARO,HK,KBSTAR,KINDEX,KODEX,KOSEF,KTOP,MASTER,...,TIGER,TIMEFOLIO,TREX,VITA,WOORI,마이다스,마이티,에셋플러스,파워,히어로즈
레버리지,,,,,,,,,,,,,,,,,,,,,
False,7.927928,0.540541,4.864865,0.540541,14.594595,9.009009,23.063063,4.324324,0.18018,0.36036,...,22.342342,0.720721,0.36036,0.18018,0.18018,0.36036,0.18018,0.36036,0.720721,0.18018
True,0.180180,0.000000,0.360360,0.000000,0.720721,0.900901,1.081081,0.720721,0.00000,0.00000,...,2.702703,0.000000,0.00000,0.00000,0.00000,0.00000,0.18018,0.00000,0.000000,0.00000


## 수치형 변수의 기술통계

In [184]:
# describe로 수치형 변수의 기술통계 구하기
desc = df.describe()


## 수치형 변수로 히스토그램 그리기

* 연속된 수치 데이터를 범주형 데이터로 표현, bucketing 이라고 부르기도 함
* 히스토그램 : 도수분포를 그래프 형태로 표현한 것
* 막대그래프와 비슷해 보이지만 다른 그래프
* 참고 : [히스토그램 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%ED%9E%88%EC%8A%A4%ED%86%A0%EA%B7%B8%EB%9E%A8)

In [185]:
# 수치형 컬럼만 리스트로 만들기
num_cols = desc.columns.tolist()
num_cols


['종목코드',
 '탭코드',
 '현재가',
 '등락구분',
 '전일비',
 '등락률',
 '순자산가치(NAV)',
 '3개월수익률',
 '거래량',
 '거래대금(백만)',
 '시가총액(억)']

In [186]:
# 히스토그램을 그려보면 수치 데이터로 되어있지만 범주형 데이터를 찾아볼 수 있습니다.
h = df[num_cols].hist(figsize=(10, 10), bins=20)
h

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f8d63ebcb10>,
      dtype=object)

## 범주형 vs 수치형 변수
### seaborn의 barplot 이해하기

In [187]:
# seaborn의 bar chart를 이용하여 각 브랜드에 대한 3개월수익률의 평균값을 볼 수 있습니다.
# 검은색 막대는 ci로 신뢰구간을 의미합니다.
sns.barplot(data=df, 
            y="브랜드", 
            x="3개월수익률", 
            orient="h", ci=None).set_title("브랜드별 평균 3개월 수익률")

Text(0.5, 1.0, '브랜드별 평균 3개월 수익률')

### barplot 그래프의 값을 groupby 로 직접 구하기

In [188]:
# 위 그래프의 평균 값을 직접 계산을 통해 구해 봅니다.
# brand_3month_mean
brand_3month_mean = df.groupby(["브랜드"])["3개월수익률"].mean()
brand_3month_mean

브랜드
ARIRANG      0.800047
FOCUS        1.791900
HANARO       0.538926
HK           1.771967
KBSTAR       0.824001
KINDEX      -2.427454
KODEX       -0.163588
KOSEF       -0.281007
KTOP        -2.822000
MASTER       2.678150
SOL         -7.465418
TIGER        0.026636
TIMEFOLIO    2.289100
TREX         0.514250
VITA              NaN
WOORI       -1.244400
마이다스         1.221850
마이티          2.828600
에셋플러스       -1.097150
파워           1.012675
히어로즈              NaN
Name: 3개월수익률, dtype: float64

In [189]:
# 위에서 구한 그룹(브랜드)별 평균 3개월 수익률을 그래프로 표현합니다.
brand_3month_mean.sort_values().plot.barh(title="브랜드별 평균 3개월 수익률")

In [190]:
# 브랜드가 마이티인 경우의 데이터를 확인합니다.
df[df["브랜드"] == "마이티"]

,종목코드,탭코드,종목명,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억),유형,브랜드,인버스,레버리지,환헤지H
438,159800,1,마이티 코스피100,26505,2,245,0.93,26602.0,-2.6910,8,0,74,국내 시장지수,마이티,False,False,False
498,292340,3,마이티 200커버드콜ATM레버리지,9250,2,165,1.82,9268.0,8.3482,285,2,56,국내 파생,마이티,False,True,False


In [191]:
# 브랜드 대비 3개월수익률에 대한 정보를 레버리지 값의 차이를 barplot으로 봅니다.
# 레버리지 유/무에 따라서 평균 3개월 수익률은 많은 차이가 없지만,
# 표준편차의 경우 레버리지가 있을경우 굉장히 편차(ci="sd")가 심함을 알 수 있습니다.
# 레버리지 상품은 오를 때 더 많이 오르고 내릴 때 더 많이 내립니다.
plt.figure(figsize=(10, 7))
sns.barplot(data=df, x="3개월수익률", y="브랜드", hue="레버리지", ci="sd")

* [Styling — pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

In [192]:
# 위에서 그린 그래프의 값을 groupby로 구합니다.
g = df.groupby(["브랜드", "레버리지"])["3개월수익률"].mean().to_frame()
g.style.background_gradient(cmap="Greens")

### boxplot으로 그리기

#### 기술통계 용어

* 백분위 수 : 데이터를 백등분 한 것
* 사분위 수 : 데이터를 4등분 한 것
* 중위수 : 데이터의 정 가운데 순위에 해당하는 값.(관측치의 절반은 크거나 같고 나머지 절반은 작거나 같다.)
* 제 3사분위 수 (Q3) : 중앙값 기준으로 상위 50% 중의 중앙값, 전체 데이터 중 상위 25%에 해당하는 값
* 제 1사분위 수 (Q1) : 중앙값 기준으로 하위 50% 중의 중앙값, 전체 데이터 중 하위 25%에 해당하는 값
* 사분위 범위 수(IQR) : 데이터의 중간 50% (Q3 - Q1)

* 출처 : [상자 수염 그림 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%83%81%EC%9E%90_%EC%88%98%EC%97%BC_%EA%B7%B8%EB%A6%BC)

In [193]:
# 브랜드별 3개월 수익률을 레버리지에 따라 다른 색상으로 시각화 합니다.
sns.boxplot(data=df, x="3개월수익률", y="브랜드", hue="레버리지")

#### boxplot 보는 방법

* 사분위수 범위(IQR, Interquartile range, Q3-Q1)
* 이상치를 제외한 최댓값  = Q3 + 1.5IQR
* 이상치를 제외한 최솟값  = Q1 - 1.5IQR

### 3개월 수익률에 대한 기술통계 값 구하기

In [194]:
# 위에서 그린 boxplot의 값을 직접 구해봅니다.
# brand_3month_desc
brand_3month_desc = df.groupby(["브랜드", "레버리지"])["3개월수익률"].describe()
brand_3month_desc

count      mean        std      min        25%      50%  \
브랜드       레버리지                                                             
ARIRANG   False   42.0  0.869126   7.087597 -22.6466  -1.221125  1.19790   
          True     1.0 -2.101300        NaN  -2.1013  -2.101300 -2.10130   
FOCUS     False    3.0  1.791900   5.295625  -1.7453  -1.252250 -0.75920   
HANARO    False   25.0  0.474488   6.704824 -16.0890  -1.897100 -0.17940   
          True     2.0  1.344400   5.368355  -2.4516  -0.553600  1.34440   
HK        False    3.0  1.771967   6.364327  -3.4743  -1.767950 -0.06160   
KBSTAR    False   79.0  1.104063   9.154995 -23.5909  -2.606150  0.20500   
          True     4.0 -4.707225  10.029810 -18.4374  -7.584975 -2.97225   
KINDEX    False   49.0 -1.841333  12.014403 -63.2817  -2.715100 -0.46210   
          True     5.0 -8.171440  16.250941 -29.8690 -20.289600 -1.83060   
KODEX     False  125.0  0.176945   9.049763 -27.3444  -3.773600  0.19870   
          True     6.0 -7.258017  16.522076 -36.8091 -11.331725 -2.60360   
KOSEF     False   23.0 -0.563013   6.213099 -10.4275  -4.301350 -0.65090   
          True     4.0  1.340525   7.931145  -7.8824  -3.627350  1.67030   
KTOP      False    1.0 -2.822000        NaN  -2.8220  -2.822000 -2.82200   
MASTER    False    2.0  2.678150   5.419337  -1.1539   0.762125  2.67815   
SOL       False   11.0 -7.465418  10.862270 -27.1984 -14.976450 -1.09590   
TIGER     False  122.0  0.854130  10.093802 -27.3958  -4.278350  0.56620   
          True    13.0 -7.739077  11.846563 -29.5670 -15.202800 -6.29080   
TIMEFOLIO False    4.0  2.289100   2.101992  -0.5080   1.260275  2.86850   
TREX      False    2.0  0.514250   2.401264  -1.1837  -0.334725  0.51425   
VITA      False    0.0       NaN        NaN      NaN        NaN      NaN   
WOORI     False    1.0 -1.244400        NaN  -1.2444  -1.244400 -1.24440   
마이다스      False    2.0  1.221850   0.931755   0.5630   0.892425  1.22185   
마이티       False    1.0 -2.691000        NaN  -2.6910  -2.691000 -2.69100   
          True     1.0  8.348200        NaN   8.3482   8.348200  8.34820   
에셋플러스     False    2.0 -1.097150   7.600337  -6.4714  -3.784275 -1.09715   
파워        False    4.0  1.012675   5.294536  -2.6130  -1.494225 -1.10965   
히어로즈      False    0.0       NaN        NaN      NaN        NaN      NaN   

                      75%      max  
브랜드       레버리지                      
ARIRANG   False  4.254325  14.0171  
          True  -2.101300  -2.1013  
FOCUS     False  3.560500   7.8802  
HANARO    False  2.511900  17.9534  
          True   3.242400   5.1404  
HK        False  4.395100   8.8518  
KBSTAR    False  4.188750  26.6595  
          True  -0.094500   5.5530  
KINDEX    False  3.370300  16.7294  
          True   2.318100   8.8139  
KODEX     False  3.993400  27.9761  
          True   1.254625  10.1686  
KOSEF     False  2.029100  13.7497  
          True   6.638175   9.9039  
KTOP      False -2.822000  -2.8220  
MASTER    False  4.594175   6.5102  
SOL       False -0.186900   3.1327  
TIGER     False  6.644800  26.2401  
          True  -0.111000   9.8470  
TIMEFOLIO False  3.897325   3.9274  
TREX      False  1.363225   2.2122  
VITA      False       NaN      NaN  
WOORI     False -1.244400  -1.2444  
마이다스      False  1.551275   1.8807  
마이티       False -2.691000  -2.6910  
          True   8.348200   8.3482  
에셋플러스     False  1.589975   4.2771  
파워        False  1.397250   8.8830  
히어로즈      False       NaN      NaN

<img src="https://miro.medium.com/max/1400/1*NRlqiZGQdsIyAu0KzP7LaQ.png" width="400">

* 이미지 출처 : https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51

### KODEX ETF 따로 분석하기

* ⚠️ 주의사항 ⚠️  분석내용은 데이터 수집 시점에 따라 달라집니다.

In [195]:
# KODEX 서브셋을 만듭니다.
kodex = df[df["브랜드"] == "KODEX"]

In [196]:
# KODEX 의 boxplot을 그립니다.
sns.boxplot(data=kodex, x="3개월수익률", y="브랜드")

In [197]:
# KODEX 의 boxenplot을 그립니다.
sns.boxenplot(data=kodex, x="3개월수익률", y="브랜드")

In [198]:
# KODEX 의 historam 을 pandas로 그립니다.
kodex["3개월수익률"].hist(bins=10)

In [199]:
# KODEX 의 kdeplot 을 그립니다.
sns.kdeplot(data=kodex, x="3개월수익률", shade=True)

In [200]:
# KODEX 의 violinplot 을 그립니다.
sns.violinplot(data=kodex, x="3개월수익률", y="브랜드", 
               cut=3, inner='quartile')

In [201]:
# KODEX 의 violinplot 에 hue="레버리지"로 그립니다.
sns.violinplot(data=kodex, x="3개월수익률", y="브랜드", hue="레버리지")

In [202]:
# KODEX 의 violinplot 에 hue="레버리지"를 split 옵션을 사용해 그립니다.
sns.violinplot(data=kodex, x="3개월수익률", y="브랜드", 
               hue="레버리지",  split=True)

In [203]:
# KODEX 의 violinplot 에 hue="인버스"로 그립니다.
sns.violinplot(data=kodex, x="3개월수익률", y="브랜드", 
               hue="인버스",  split=True)

* 이미지 출처 : [The Datasaurus Dozen - Same Stats, Different Graphs | Autodesk Research](https://www.autodeskresearch.com/publications/samestats)

<img src="https://d2f99xq7vri1nk.cloudfront.net/BoxViolinSmaller.gif">

### 브랜드별 3개월수익률

In [204]:
# boxenplot 을 통해 브랜드별 3개월 수익률을 시각화 합니다.
sns.boxenplot(data=df, x="3개월수익률", y="브랜드")

In [205]:
# violinplot은 kdeplot을 마주보도록 그린 그래프로 값의 분포를 알 수 있습니다.
# boxplot의 단점을 보완하기 위해 만들어지기도 했는데 내부에 boxplot을 함께 그려줍니다.
# 가운데 점은 중앙값을 의미합니다.
sns.violinplot(data=df, x="3개월수익률", y="브랜드")

### 서브플롯 그려보기

In [206]:
# boxplot과 violinplot을 하나의 그래프에 같이 그려봅니다.
fig, axes = plt.subplots(ncols=2, figsize=(15, 4))
sns.boxplot(data=df, x="3개월수익률", y="브랜드", ax=axes[0])
sns.violinplot(data=df, x="3개월수익률", y="브랜드", ax=axes[1])

In [207]:
# violinplot은 kdeplot을 마주보도록 그린 그래프로 값의 분포를 알 수 있습니다.
# boxplot의 단점을 보완하기 위해 만들어지기도 했는데 내부에 boxplot을 함께 그려줍니다.
# 가운데 점은 중앙값을 의미합니다.
# 브랜드별 3개월수익률을 레버리지 여부에 따라 그려봅니다.
plt.figure(figsize=(10, 6))
sns.violinplot(data=df, x="3개월수익률", y="브랜드", hue="레버리지", split=True)

In [208]:
# 브랜드별 3개월수익률을 인버스 여부에 따라 그려봅니다.
plt.figure(figsize=(10, 6))
sns.violinplot(data=df, x="3개월수익률", y="브랜드", hue="인버스", split=True)

### 브랜드별 거래량

In [209]:
# 수집한 날짜에 대한 브랜드별 거래량을 봅니다.
sns.barplot(data=df, x="거래량", y="브랜드", ci=None)

In [210]:
df[df["브랜드"].isin(["TIGER", "KBSTAR", "KINDEX"])]

,종목코드,탭코드,종목명,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억),유형,브랜드,인버스,레버리지,환헤지H
1,371460,4,TIGER 차이나전기차SOLACTIVE,11890,2,95,0.81,11894.0,-25.5132,8956866,107221,29425,해외 주식,TIGER,False,False,False
3,133690,4,TIGER 미국나스닥100,75220,2,1000,1.35,74008.0,-0.9795,117485,8789,20964,해외 주식,TIGER,False,False,False
5,102110,1,TIGER 200,35060,2,330,0.95,35137.0,-0.3681,1568552,54652,18126,국내 시장지수,TIGER,False,False,False
6,157450,6,TIGER 단기통안채,100790,2,15,0.01,100796.0,0.2187,976423,98417,18098,채권,TIGER,False,False,False
8,360750,4,TIGER 미국S&P500,13510,2,155,1.16,13364.0,3.3166,665897,8935,16097,해외 주식,TIGER,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,295000,6,KBSTAR 국채선물10년,51165,5,-10,-0.02,51181.0,-3.7159,39,1,20,채권,KBSTAR,False,False,False
551,287330,2,KBSTAR 200생활소비재,7395,2,255,3.57,7392.0,6.9597,485,3,18,국내 업종/테마,KBSTAR,False,False,False
552,284990,2,KBSTAR 200에너지화학,11195,2,290,2.66,11214.0,1.1791,316,3,16,국내 업종/테마,KBSTAR,False,False,False
553,287310,2,KBSTAR 200경기소비재,10060,2,100,1.00,10079.0,2.7763,380,3,10,국내 업종/테마,KBSTAR,False,False,False


In [211]:
# 전체 브랜드를 보면 값의 크기 차이로 시각화를 제대로 보기 어렵습니다.
# 일부 브랜드 "TIGER", "KBSTAR", "KINDEX" 를 따로 가져와서 시각화 합니다.
sns.barplot(data=df[df["브랜드"].isin(["TIGER", "KBSTAR", "KINDEX"])], 
            y="브랜드", x="거래량")

In [212]:
sns.violinplot(data=df[df["브랜드"].isin(["TIGER", "KBSTAR", "KINDEX"])], 
            y="브랜드", x="거래량", hue="레버리지", split=True)

In [213]:
# "마이티", "마이다스", "흥국" 만 봅니다.
sns.violinplot(data=df[df["브랜드"].isin(["마이티", "마이다스", "흥국"])], 
            y="브랜드", x="거래량", hue="레버리지", split=True)

In [214]:
# "마이티", "마이다스", "흥국" 의 기술통계 값 보기
df[df["브랜드"].isin(["마이티", "마이다스", "흥국"])].groupby("브랜드")["거래량"].describe()

,count,mean,std,min,25%,50%,75%,max
브랜드,,,,,,,,
마이다스,2.0,50.5,65.760931,4.0,27.25,50.5,73.75,97.0
마이티,2.0,146.5,195.868578,8.0,77.25,146.5,215.75,285.0


### 유형에 따른 거래량

In [215]:
# 유형에 따른 평균 거래량을 barplot으로 시각화 합니다.
plt.figure(figsize=(10, 4))
sns.barplot(data=df, x="유형", y="거래량", 
            ci=None).set_title("ETF 유형별 평균 거래량")

Text(0.5, 1.0, 'ETF 유형별 평균 거래량')

In [216]:
# 위 그래프의 유형에 따른 평균 거래량을 직접 groupby를 통해 구합니다.
df.groupby(["유형"])["거래량"].mean().sort_values(ascending=False)

유형
국내 파생       6.383260e+06
국내 시장지수     1.873465e+05
원자재         1.862468e+05
해외 주식       1.471746e+05
국내 업종/테마    1.085171e+05
채권          7.927155e+04
기타          4.086323e+04
Name: 거래량, dtype: float64

### 유형에 따른 거래대금

In [217]:
# 위 그래프의 유형에 따른 평균 거래대금(백만)을 barplot 으로 그립니다.
plt.figure(figsize=(10, 4))
sns.barplot(data=df, x="유형", y="거래대금(백만)", 
            ci=None).set_title("ETF유형별 평균 거래대금(백만)")

Text(0.5, 1.0, 'ETF유형별 평균 거래대금(백만)')

In [218]:
# 위 그래프의 유형에 따른 평균 거래대금(백만)을 직접 groupby를 통해 구합니다.
df.groupby(["유형"])["거래대금(백만)"].mean().sort_values(ascending=False)

유형
국내 파생       29242.648649
채권           7635.633333
국내 시장지수      4912.796875
해외 주식        1652.764706
국내 업종/테마     1139.019512
원자재          1075.722222
기타            383.542857
Name: 거래대금(백만), dtype: float64

In [219]:
# 위 그래프의 유형에 따른 평균 거래대금(백만)을 직접 pivot_table을 통해 구합니다.
pd.pivot_table(data=df, index="유형", 
               values="거래대금(백만)").sort_values("거래대금(백만)", ascending=False)

,거래대금(백만)
유형,
국내 파생,29242.648649
채권,7635.633333
국내 시장지수,4912.796875
해외 주식,1652.764706
국내 업종/테마,1139.019512
원자재,1075.722222
기타,383.542857


### 유형에 따른 시가총액

In [220]:
# 유형에 따른 평균 시가총액(억) 에 대한 시각화를 합니다.
plt.figure(figsize=(10, 4))
sns.barplot(data=df, x="유형", y="시가총액(억)", 
            ci=None).set_title("ETF유형별 평균 시가총액(억)")

Text(0.5, 1.0, 'ETF유형별 평균 시가총액(억)')

In [221]:
# 위 그래프의 유형에 따른 평균 시가총액(억)을 직접 groupby를 통해 구합니다.
df.groupby("유형")["시가총액(억)"].mean().sort_values(ascending=False)

유형
국내 시장지수     2658.703125
국내 파생       1919.837838
채권          1912.883333
해외 주식       1387.080882
국내 업종/테마     795.970732
원자재          503.222222
기타           393.400000
Name: 시가총액(억), dtype: float64

In [222]:
# 위 그래프의 유형에 따른 평균 시가총액(억)을 직접 pivot_table을 통해 구합니다.
pd.pivot_table(data=df, index="유형", 
               values="시가총액(억)").sort_values(by="시가총액(억)", ascending=False)

,시가총액(억)
유형,
국내 시장지수,2658.703125
국내 파생,1919.837838
채권,1912.883333
해외 주식,1387.080882
국내 업종/테마,795.970732
원자재,503.222222
기타,393.400000


### 유형에 따른 3개월 수익률

In [223]:
# 유형에 따른 평균 3개월수익률에 대한 시각화를 합니다.
plt.figure(figsize=(10, 4))
sns.barplot(data=df, x="유형", y="3개월수익률", 
            ci=None).set_title("ETF유형별 3개월수익률")

Text(0.5, 1.0, 'ETF유형별 3개월수익률')

In [224]:
# 위 그래프의 유형에 따른 평균 3개월수익률을 직접 groupby를 통해 구합니다.
df.groupby("유형")["3개월수익률"].mean().sort_values(ascending=False)

유형
원자재         4.414328
국내 업종/테마    3.244097
기타          0.957938
국내 시장지수     0.427703
국내 파생      -0.092365
채권         -1.111405
해외 주식      -6.390003
Name: 3개월수익률, dtype: float64

In [225]:
# 위 그래프의 유형에 따른 평균 3개월수익률을 직접 pivot_table을 통해 구합니다.
pd.pivot_table(data=df, index="유형", 
               values="3개월수익률").sort_values("3개월수익률", ascending=False)

,3개월수익률
유형,
원자재,4.414328
국내 업종/테마,3.244097
기타,0.957938
국내 시장지수,0.427703
국내 파생,-0.092365
채권,-1.111405
해외 주식,-6.390003


## 여러 개의 변수로 서브플롯 그리기
### PairGrid와 barplot 으로 서브플롯 그리기


* PairGrid는 한번에 서브플롯을 쉽게 그려볼 수 있는 장점이 있지만 아쉽게도 share axis in PairGrid는 지원하지 않고 있습니다.
* 참고 : https://github.com/mwaskom/seaborn/issues/1032

In [226]:
# PairGrid 와 barplot을 한번에 서브플롯으로 그립니다.
# 유형에 따른 "거래량", "거래대금(백만)", "시가총액(억)", "3개월수익률" 에 대한 시각화를 합니다.
g = sns.PairGrid(data=df, 
             x_vars="유형", 
             y_vars=["거래량", "거래대금(백만)", "시가총액(억)", "3개월수익률"], 
             aspect=5)
g.map(sns.barplot)
g.set(xlabel="시장구분")

### subplots를 통한 시각화

In [227]:
row_cols = ["거래량", "거래대금(백만)", "시가총액(억)", "3개월수익률"]

In [228]:
# 위 그래프를 직접 subplots 를 사용해 그려봅니다.
fig, axes = plt.subplots(nrows=len(row_cols), figsize=(10, 12))
for i, rcol in enumerate(row_cols):
    sns.barplot(data=df, x="유형", y=rcol, ax=axes[i], ci=None)

In [229]:
# 브랜드별 수치 데이터의 평균을 시각화 합니다.
fig, axes = plt.subplots(nrows=len(row_cols), figsize=(15, 20))
for i, rcol in enumerate(row_cols):
    sns.barplot(data=df, x="브랜드", y=rcol, ax=axes[i], ci=None).set_title(f"평균 {rcol}")

### 브랜드별 인버스 여부에 따른 시각화

In [230]:
# subplots를 시각화 합니다. hue="인버스"를 적용해 봅니다.
fig, axes = plt.subplots(nrows=len(row_cols), figsize=(15, 20))
for i, rcol in enumerate(row_cols):
    g = sns.barplot(data=df, x="브랜드", y=rcol, hue="인버스",
                ax=axes[i], ci=None)
    g.set_title(f"평균 {rcol}")
    g.legend(loc=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


### 브랜드별 레버리지 여부에 따른 시각화

In [231]:
# subplots를 시각화 합니다. hue="레버리지"를 적용해 봅니다.
fig, axes = plt.subplots(nrows=len(row_cols), figsize=(15, 20))
for i, rcol in enumerate(row_cols):
    g = sns.barplot(data=df, x="브랜드", y=rcol, hue="레버리지",
                ax=axes[i], ci=None)
    g.set_title(f"평균 {rcol}")
    g.legend(loc=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


In [232]:
# 위에서 그린 서브플롯을 직접 groupby 로 구해봅니다.
gb = df.groupby(["브랜드", "레버리지"])[row_cols].mean()
gb.style.background_gradient()

### 유형에 따른 3개월 수익률 boxplot 
* barplot으로 그렸을 때 보이지 않던 데이터의 빈도수와 데이터의 분포를 확인할 수 있습니다.


In [233]:
# boxplot 으로 x축에 "유형" y축에 "3개월수익률" 을 그려봅니다.
plt.figure(figsize=(10, 4))
sns.boxplot(data=df, x="유형", y="3개월수익률")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


In [234]:
# boxplot의 유형에 따른 3개월 수익률에 대한 기술통계 값을 구합니다.
df.groupby(["유형"])["3개월수익률"].describe()

,count,mean,std,min,25%,50%,75%,max
유형,,,,,,,,
국내 시장지수,64.0,0.427703,3.109923,-3.5516,-1.119425,-0.43670,1.511975,11.1225
국내 업종/테마,199.0,3.244097,7.418752,-17.5480,-1.144500,2.51190,7.014450,22.9746
국내 파생,37.0,-0.092365,4.263917,-6.2441,-2.209000,-0.52730,0.635200,14.0798
기타,34.0,0.957938,5.978500,-9.8419,-1.549250,0.93855,5.138400,10.1686
원자재,18.0,4.414328,16.206221,-27.3958,-4.111375,3.97325,15.803200,27.9761
채권,59.0,-1.111405,5.828818,-18.4374,-2.841050,-1.12130,0.301300,20.5787
해외 주식,126.0,-6.390003,13.048848,-63.2817,-13.723400,-6.46025,2.795600,26.6595


### subplots 로 유형에 대한 boxplot 그리기

In [235]:
for i, rcol in enumerate(row_cols):
    print(i, rcol)

0 거래량
1 거래대금(백만)
2 시가총액(억)
3 3개월수익률


In [236]:
# subplots 를 통한 유형에 따른 수치데이터의 평균값 구하기
fig, axes = plt.subplots(nrows=len(row_cols), figsize=(10, 10))
for i, rcol in enumerate(row_cols):
    sns.boxplot(data=df, x="유형", y=rcol, ax=axes[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


In [237]:
# hue="레버리지" 를 추가해서 그립니다.

fig, axes = plt.subplots(nrows=len(row_cols), figsize=(10, 10))
for i, rcol in enumerate(row_cols):
    sns.boxplot(data=df, x="유형", y=rcol, ax=axes[i], hue="레버리지")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


### subplots 로 유형에 대한  violinplot 그리기
* boxplot으로 그렸을 때 박스 안의 분포를 좀 더 자세히 그려줍니다.

In [238]:
# 유형에 따른 violinplot 

fig, axes = plt.subplots(nrows=len(row_cols), figsize=(10, 10))
for i, rcol in enumerate(row_cols):
    sns.violinplot(data=df, x="유형", y=rcol, ax=axes[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


In [239]:
# 유형에 따른 violinplot 에 hue="레버리지" 적용하기

fig, axes = plt.subplots(nrows=len(row_cols), figsize=(10, 10))
for i, rcol in enumerate(row_cols):
    sns.violinplot(data=df, x="유형", y=rcol, ax=axes[i], hue="레버리지", split=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


### PairGrid 를 통해 여러 변수의 violinplot 그리기

In [240]:
# 브랜드에 따른 여러 수치변수의 violinplot 를 PairGrid로 그리기
g = sns.PairGrid(data=df, x_vars="브랜드", y_vars=row_cols, aspect=5, hue="레버리지")
g.map(sns.violinplot)

/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:1242: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


### subplots 로 브랜드에 대한 boxplot 그리기

In [241]:
# 브랜드에 따른 여러 수치변수의 boxplot 을 subplots으로 그리기 : hue="레버리지"
fig, axes = plt.subplots(nrows=len(row_cols), figsize=(10, 10))
for i, rcol in enumerate(row_cols):
    sns.boxplot(data=df, x="브랜드", y=rcol, ax=axes[i], hue="레버리지")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


### subplots 로 브랜드에 대한 violinplot 그리기

In [242]:
# 브랜드에 따른 여러 수치변수의 violinplot 을 subplots으로 그리기 : hue="레버리지"

fig, axes = plt.subplots(nrows=len(row_cols), figsize=(10, 10))
for i, rcol in enumerate(row_cols):
    sns.violinplot(data=df, x="브랜드", y=rcol, ax=axes[i], hue="레버리지", split=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


## FacetGrid 를 통한 범주형 변수에 대한 서브플롯 그리기

In [243]:
# catplot 을 통한 브랜드별 3개월 수익률
sns.catplot(data=df, x="브랜드", y="3개월수익률", aspect=3)

/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


In [244]:
# catplot 을 통한 브랜드별 3개월 수익률 : kind="bar"

sns.catplot(data=df, x="브랜드", y="3개월수익률", aspect=3, kind="bar", ci=None)

/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


In [245]:
# catplot 을 통한 브랜드별 3개월 수익률 : kind="box"

sns.catplot(data=df, x="브랜드", y="3개월수익률", aspect=3, kind="box", ci=None)

/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


In [246]:
# catplot 을 통한 브랜드별 3개월 수익률 : kind="violin"

sns.catplot(data=df, x="브랜드", y="3개월수익률", aspect=3, kind="violin", ci=None)

/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


### 특정 범주형 변수에 대한 서브플롯 그리기 - catplot
#### 브랜드 별 빈도수 - countplot

In [247]:
sns.catplot(data=df, y="브랜드", kind="count", sharex=False,
            col="유형", col_wrap=2)

/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


#### 브랜드별 3개월 수익률 - barplot

In [248]:
sns.catplot(data=df, y="브랜드", x="3개월수익률", kind="bar", sharex=False,
            col="유형", col_wrap=2, ci=None)

/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


#### 브랜드별 거래량 - barplot

In [249]:
sns.catplot(data=df, y="브랜드", x="거래량", kind="bar", sharex=False,
            col="유형", col_wrap=2, ci=None)

/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)


## 해외 주식 ETF

In [250]:
# 유형이 "해외 주식"인 ETF만 가져옵니다.
oversea = df[df["유형"] == "해외 주식"]

In [251]:
# "해외 주식"ETF의 브랜드별 "3개월수익률"을 시각화 합니다.
plt.figure(figsize=(10, 4))
sns.barplot(data=oversea, x="브랜드", y="3개월수익률", ci=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


In [252]:
# "해외 주식"ETF의 브랜드별 평균 "3개월수익률"을 hue="환헤지H"로 시각화 합니다.

plt.figure(figsize=(10, 4))
sns.barplot(data=oversea, x="브랜드", y="3개월수익률", ci=None, hue="환헤지H")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


In [253]:
# "해외 주식"ETF의 브랜드별 평균 "3개월수익률"을 violinplot으로 시각화 합니다.

plt.figure(figsize=(10, 4))
sns.violinplot(data=oversea, x="브랜드", y="3개월수익률", ci=None, hue="환헤지H", split=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


In [254]:
# 위에서 그린 그래프를 catplot을 통해 그리며  col="레버리지", row="인버스"로 시각화 합니다.
sns.catplot(data=oversea, x="브랜드", y="3개월수익률", kind="violin", sharex=False,
            col="레버리지", row="인버스")

/usr/local/lib/python3.7/dist-packages/seaborn/categorical.py:3806: UserWarning: Setting `sharex=False` with `color=None` may cause different levels of the `x` variable to share colors. This will change in a future version.
  warnings.warn(msg.format("sharex", "x"), UserWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/axisgrid.py:409: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)



## 상관분석

상관 분석(相關 分析, Correlation analysis) 또는 '상관관계' 또는 '상관'은 확률론과 통계학에서 두 변수간에 어떤 선형적 또는 비선형적 관계를 갖고 있는지를 분석하는 방법이다. 두 변수는 서로 독립적인 관계이거나 상관된 관계일 수 있으며 이때 두 변수간의 관계의 강도를 상관관계(Correlation, Correlation coefficient)라 한다. 상관분석에서는 상관관계의 정도를 나타내는 단위로 모상관계수로 ρ를 사용하며 표본 상관 계수로 r 을 사용한다.

상관관계의 정도를 파악하는 상관 계수(相關係數, Correlation coefficient)는 두 변수간의 연관된 정도를 나타낼 뿐 인과관계를 설명하는 것은 아니다. 두 변수간에 원인과 결과의 인과관계가 있는지에 대한 것은 회귀분석을 통해 인과관계의 방향, 정도와 수학적 모델을 확인해 볼 수 있다.

* 출처 : https://ko.wikipedia.org/wiki/%EC%83%81%EA%B4%80_%EB%B6%84%EC%84%9D

### 피어슨 상관계수

* r 값은 X 와 Y 가 완전히 동일하면 +1, 전혀 다르면 0, 반대방향으로 완전히 동일 하면 –1 을 가진다. 
* 결정계수(coefficient of determination)는 r^2 로 계산하며 이것은 X 로부터 Y 를 예측할 수 있는 정도를 의미한다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Correlation_examples2.svg/800px-Correlation_examples2.svg.png" width="500">

```
일반적으로
r이 -1.0과 -0.7 사이이면, 강한 음적 선형관계,
r이 -0.7과 -0.3 사이이면, 뚜렷한 음적 선형관계,
r이 -0.3과 -0.1 사이이면, 약한 음적 선형관계,
r이 -0.1과 +0.1 사이이면, 거의 무시될 수 있는 선형관계,
r이 +0.1과 +0.3 사이이면, 약한 양적 선형관계,
r이 +0.3과 +0.7 사이이면, 뚜렷한 양적 선형관계,
r이 +0.7과 +1.0 사이이면, 강한 양적 선형관계
```

<img src="https://upload.wikimedia.org/wikipedia/commons/3/34/Correlation_coefficient.png" width=500>

* 출처 :  https://ko.wikipedia.org/wiki/%EC%83%81%EA%B4%80_%EB%B6%84%EC%84%9D#%ED%94%BC%EC%96%B4%EC%8A%A8_%EC%83%81%EA%B4%80_%EA%B3%84%EC%88%98



### 상관계수 구하기

In [258]:
# 전체 데이터프레임에 대한 상관계수를 구합니다.
corr = df.corr()

In [259]:
# np.triu : matrix를 상삼각행렬로 만드는 numpy math
# [1 2 3]   np.triu  [1 2 3]
# [4 5 6]   -------> [0 5 6]
# [2 3 4]            [0 0 4]
# np.ones_like(x) : x와 크기만 같은 1로 이루어진 array를 만듬

# 수식적으로 어려워 보일수도 있지만 간단함
# 자기상관계수는 대각행렬을 기준으로 대칭되어 같은 값이 출력되므로,
# 이대로 전체를 heatmap을 plot하면 오히려 가독성이 떨어질 수 있음
# 이에, 가독성을 높이기 위해 대각행렬 기준으로 한쪽의 데이터들만 masking 기법을 통해 plot하여 
# 가독성을 높이는 효과를 가질수 있음
# np.ones_like로 heatmap의 마스크값 구하기
mask = np.triu(np.ones_like(corr))

In [260]:
# seaborn 의 heatmap으로 상관 계수 시각화 하기
plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", vmax=1, vmin=-1, mask=mask)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


### 양 혹은 음의 상관 계수가 높은 변수끼리의 regplot 그리기

In [261]:
# regplot을 통해 양의 상관 계수가 높은 변수끼리 시각화 합니다.
sns.regplot(data=df, x="전일비", y="등락률", ci=None)

In [262]:
# regplot을 통해 음의 상관 계수가 높은 변수끼리 시각화 합니다.
sns.regplot(data=df, x="등락률", y="등락구분")

### 종목별 상관 계수

In [263]:
# select_dtypes 를 통해 object 타입을 제외한 타입을 제외하고 가져오록 합니다.
# 종목명을 인덱스로 만들어 줍니다.
df_num = df.select_dtypes(exclude=np.object).copy()
df_num["종목명"] = df["종목명"]
df_num = df_num.set_index("종목명")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [264]:
# transpose 로 행과 열을 바꿉니다.
df_t = df_num.T

In [265]:
# df_t의 데이터 타입을 float 으로 변경하고 상관 계수 구하기
etf_corr = df_t.astype(float).corr()
etf_corr

종목명,KODEX 200,TIGER 차이나전기차SOLACTIVE,KODEX 레버리지,TIGER 미국나스닥100,KODEX 200선물인버스2X,TIGER 200,TIGER 단기통안채,KODEX 단기채권PLUS,TIGER 미국S&P500,KODEX 삼성그룹,...,KBSTAR 200철강소재,ARIRANG KRX300헬스케어,ARIRANG 코스닥150,TIGER 가격조정,KBSTAR 모멘텀밸류,KBSTAR 국채선물10년,KBSTAR 200생활소비재,KBSTAR 200에너지화학,KBSTAR 200경기소비재,KBSTAR 200산업재
종목명,,,,,,,,,,,,,,,,,,,,,
KODEX 200,1.000000,0.999311,0.999798,0.531216,0.999410,0.998741,0.982752,0.371222,0.877584,0.947836,...,-0.041066,-0.054666,-0.071555,-0.081988,-0.074118,-0.091084,-0.067959,-0.070666,-0.069727,-0.071888
TIGER 차이나전기차SOLACTIVE,0.999311,1.000000,0.999379,0.545440,0.999216,0.999273,0.983507,0.388705,0.890910,0.956336,...,-0.012777,-0.026363,-0.043383,-0.054556,-0.046082,-0.064649,-0.039639,-0.042472,-0.041495,-0.043712
KODEX 레버리지,0.999798,0.999379,1.000000,0.522596,0.999874,0.998139,0.979987,0.362214,0.874632,0.945660,...,-0.047326,-0.060897,-0.077874,-0.088796,-0.080543,-0.098349,-0.074153,-0.076968,-0.075997,-0.078203
TIGER 미국나스닥100,0.531216,0.545440,0.522596,1.000000,0.519879,0.570889,0.668470,0.980317,0.787426,0.725920,...,0.653028,0.643033,0.640735,0.675567,0.648298,0.706277,0.632423,0.640054,0.637777,0.640466
KODEX 200선물인버스2X,0.999410,0.999216,0.999874,0.519879,1.000000,0.997580,0.978189,0.358975,0.872715,0.944476,...,-0.051207,-0.064778,-0.081738,-0.092549,-0.084386,-0.101967,-0.078036,-0.080833,-0.079869,-0.082066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KBSTAR 국채선물10년,-0.091084,-0.064649,-0.098349,0.706277,-0.101967,-0.041579,0.059130,0.825301,0.387109,0.223901,...,0.980801,0.980511,0.983748,0.995830,0.986947,1.000000,0.979755,0.983299,0.982236,0.983733
KBSTAR 200생활소비재,-0.067959,-0.039639,-0.074153,0.632423,-0.078036,-0.020883,0.061792,0.751193,0.418097,0.249622,...,0.999580,0.999893,0.999778,0.993928,0.999206,0.979755,1.000000,0.999828,0.999918,0.999780
KBSTAR 200에너지화학,-0.070666,-0.042472,-0.076968,0.640054,-0.080833,-0.023287,0.061199,0.759119,0.415530,0.247228,...,0.999530,0.999795,0.999996,0.995798,0.999773,0.983299,0.999828,1.000000,0.999983,0.999997


* "KODEX 200" 과 상관계수가 높은 ETF 상위 10개

In [266]:
# "KODEX 200" 과 상관계수가 높은 ETF 상위 10개
etf_corr["KODEX 200"].sort_values(ascending=False).head(20)

종목명
KODEX 200                1.000000
KODEX 레버리지               0.999798
KODEX 코스닥150레버리지         0.999535
KODEX 코스닥150선물인버스        0.999517
KODEX 인버스                0.999480
KODEX 200선물인버스2X         0.999410
TIGER 200선물인버스2X         0.999400
TIGER 차이나전기차SOLACTIVE    0.999311
TIGER 200                0.998741
KODEX 코스닥150             0.998729
KODEX 은행                 0.998349
KODEX 단기채권               0.996610
TIGER 인버스                0.995753
KOSEF 200                0.994069
TIGER 화장품                0.993762
TIGER 200 중공업            0.990503
KODEX 2차전지산업             0.989010
KODEX 건설                 0.986354
TIGER 2차전지테마             0.985122
KODEX 자동차                0.983306
Name: KODEX 200, dtype: float64

* "KODEX 200" 과 상관계수가 낮은 ETF 하위 10개

In [267]:
# "KODEX 200" 과 상관계수가 낮은 ETF 하위 10개

etf_corr["KODEX 200"].sort_values().head(10)

종목명
KINDEX 국고채3년            -0.134069
KOSEF 국고채3년             -0.134061
KBSTAR 국고채3년            -0.132632
KOSEF 통안채1년             -0.131181
KBSTAR 중기우량회사채          -0.130228
KOSEF 국고채10년레버리지        -0.129081
파워 중기국고채                -0.124980
KOSEF 단기자금              -0.124548
KINDEX 단기통안채            -0.122210
KINDEX 미국다우존스리츠(합성 H)   -0.119898
Name: KODEX 200, dtype: float64

### 2X ETF의 상관계수

In [268]:
# 2X가 들어가는 ETF의 상관 계수만 가져오기
etf_corr_2x = etf_corr.filter(regex="2X").filter(regex="2X", axis=0)
etf_corr_2x

종목명,KODEX 200선물인버스2X,TIGER 200선물인버스2X,KBSTAR 200선물인버스2X,KODEX 미국달러선물인버스2X,KOSEF 미국달러선물인버스2X,ARIRANG 200선물인버스2X,KOSEF 200선물인버스2X,TIGER 미국달러선물인버스2X,KBSTAR 미국장기국채선물인버스2X(합성 H)
종목명,,,,,,,,,
KODEX 200선물인버스2X,1.000000,0.999775,0.634749,0.914407,0.112302,0.166275,0.074221,-0.026015,-0.069406
TIGER 200선물인버스2X,0.999775,1.000000,0.650983,0.922778,0.133329,0.187134,0.095339,-0.004830,-0.048255
KBSTAR 200선물인버스2X,0.634749,0.650983,1.000000,0.893100,0.838869,0.867425,0.817699,0.755694,0.726572
KODEX 미국달러선물인버스2X,0.914407,0.922778,0.893100,1.000000,0.504901,0.551161,0.471404,0.380844,0.340327
KOSEF 미국달러선물인버스2X,0.112302,0.133329,0.838869,0.504901,1.000000,0.998457,0.998953,0.990416,0.983482
ARIRANG 200선물인버스2X,0.166275,0.187134,0.867425,0.551161,0.998457,1.000000,0.995599,0.981360,0.972112
KOSEF 200선물인버스2X,0.074221,0.095339,0.817699,0.471404,0.998953,0.995599,1.000000,0.994653,0.989394
TIGER 미국달러선물인버스2X,-0.026015,-0.004830,0.755694,0.380844,0.990416,0.981360,0.994653,1.000000,0.999056
KBSTAR 미국장기국채선물인버스2X(합성 H),-0.069406,-0.048255,0.726572,0.340327,0.983482,0.972112,0.989394,0.999056,1.000000


In [269]:
# np.ones_like로 heatmap의 마스크값 구하기
mask_2x = np.triu(np.ones_like(etf_corr_2x))

In [270]:
# seaborn 의 heatmap으로 상관 계수 시각화 하기
plt.figure(figsize=(8, 8))
sns.heatmap(etf_corr_2x, annot=True, cmap="coolwarm", vmax=1, vmin=-1, mask=mask_2x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


### 원유와 관련된 ETF의 상관계수 보기

In [271]:
# 원유가 들어가는 상관계수만 가져오기
etf_corr_oil = etf_corr.filter(regex="원유").filter(regex="원유", axis=0)

In [272]:
# np.ones_like로 heatmap의 마스크값 구하기
mask_oil = np.triu(np.ones_like(etf_corr_oil))

In [273]:
# seaborn 의 heatmap으로 상관 계수 시각화 하기

plt.figure(figsize=(8, 8))
sns.heatmap(etf_corr_oil, annot=True, cmap="coolwarm", vmax=1, vmin=-1, mask=mask_oil)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


## seaborn 공식문서

* seaborn 에 대해 좀 더 알고 싶다면 다음의 튜토리얼을 참고해 보세요.
* [User guide and tutorial — seaborn documentation](https://seaborn.pydata.org/tutorial.html)